# Price calculator

This file shows an example of how to calculate the price of openai API calls with our pricing toolkit.

In [29]:
from utils import get_openai_api_key, get_openai_base_url
import openai
from pprint import pprint
import json
from typing import List, Dict, Optional, Any, Union

In [30]:
def chat_demo(model: str, messages: List[Dict[str, Any]]) -> Dict[str, Any]:
    api_keys = get_openai_api_key()
    base_url = get_openai_base_url()

    client = openai.OpenAI(api_key=api_keys, base_url=base_url)
    response = client.chat.completions.create(model=model, messages=messages)
    
    return response

In [35]:

model = "gpt-3.5-turbo"
message = [
    {
        "role": "user",
        "content": "Translate the following English text to French: 'Hello, how are you?'",
    }
]

response = chat_demo(model=model, messages=message)

print(f"Response:\n{response}")
pprint(response.to_dict())

Response:
ChatCompletion(id='chatcmpl-AnHWjfrVYkHEzsy0TxcAEb7oraCVT', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Salut, comment vas-tu?', refusal=None, role='assistant', audio=None, function_call=None, tool_calls=None))], created=1736308809, model='gpt-3.5-turbo-0125', object='chat.completion', service_tier=None, system_fingerprint='fp_0165350fbb', usage=CompletionUsage(completion_tokens=8, prompt_tokens=22, total_tokens=30, completion_tokens_details=None, prompt_tokens_details=None))
{'choices': [{'finish_reason': 'stop',
              'index': 0,
              'logprobs': None,
              'message': {'content': 'Salut, comment vas-tu?',
                          'role': 'assistant'}}],
 'created': 1736308809,
 'id': 'chatcmpl-AnHWjfrVYkHEzsy0TxcAEb7oraCVT',
 'model': 'gpt-3.5-turbo-0125',
 'object': 'chat.completion',
 'system_fingerprint': 'fp_0165350fbb',
 'usage': {'completion_tokens': 8, 'prompt_tokens': 22, 'total_toke

In [36]:

def track_costs(response: openai.ChatCompletion) -> Dict[str, Any]:
    # load price data: $/1k tokens
    with open('price_usd_per_1k_tokens.json') as f:
        price = json.load(f)

    model_name = response.model
    usage = response.usage.to_dict()

    model_price = price.get(model_name, {})
    input_cost = (
        model_price.get("input_tokens", 0) * usage.get("prompt_tokens", 0) / 1000
    )
    output_cost = (
        model_price.get("output_tokens", 0) * usage.get("completion_tokens", 0) / 1000
    )
    total_cost = input_cost + output_cost

    print(f"The total cost for using {model_name} is ${total_cost:.5f}")
    return total_cost

track_costs(response)

The total cost for using gpt-3.5-turbo-0125 is $0.00000


0.0